# A train ride through the Netherlands

In this notebook we will use the API of the Dutch Railways to find the following information from each Dutch station: 

* Stationcode;
* Latitude;
* Longitude;
* Short name;
* Average name;
* Long name;
* Final destination;
* Earliest time of departure.

We'll save this information in a mySQL database. I will follow various steps to achieve my goal. I'll first import the request-module, so that I can connect to the API I need first, the travel information of the NS (Dutch Railways) with the category 'stations'. 

In [ ]:
import requests
my_headers = {'Ocp-Apim-Subscription-Key': 'secret-secret-secret'}
r = requests.get('https://gateway.apiportal.ns.nl/reisinformatie-api/api/v2/stations', headers=my_headers)

## Step 1: station data

With this data I can find the first 6 items on my list of the 400 Dutch stations. I'll transform the information to a JSON-object so that I can process it easier, and I'll look at one item first to see what information I'm receiving. 

In [2]:
data = r.json()

data['payload'][0]

{'UICCode': '8002084',
 'stationType': 'SNELTREIN_STATION',
 'EVACode': '8000139',
 'code': 'MGZB',
 'sporen': [],
 'synoniemen': ['Gunzburg'],
 'heeftFaciliteiten': True,
 'heeftVertrektijden': True,
 'heeftReisassistentie': False,
 'namen': {'lang': 'Günzburg', 'middel': 'Günzburg', 'kort': 'Günzburg'},
 'land': 'D',
 'lat': 48.460226,
 'lng': 10.278707,
 'radius': 0,
 'naderenRadius': 0,
 'ingangsDatum': '2018-12-16',
 'nearbyMeLocationId': {'value': 'MGZB', 'type': 'stationV2'}}

I can see the information I need, but I also notice I'm receiving foreign stations. I'll have to filter for 'land': 'NL'. 

I'll make six lists to store my information. 
I'll save data[payload] as a subset of my data, considering it has everything I need. It'll be easier to use and I won't have to type it out every time. 

Then I'll go through th elocations to save the correct information. I'll make a variable 'num'. I'll put a number behind my dataset, starting with the first item and number 0. If I have the information from this location, I'll add 1 to the number, and restart the loop so it will go to the next location. 

I'll finish by printing one of my lists, to check if it only has Dutch stations, and by printing the length of the list, to see if the amount is sensible.

In [3]:
stationscodes = []
latitudes = []
longitudes = []
naam_kort = []
naam_mid = []
naam_lang = []

dataset = data['payload']
num = 0
for item in dataset:
    if dataset[num]['land'] == 'NL':
        stationscodes.append(dataset[num]['code'])
        latitudes.append(dataset[num]['lat'])
        longitudes.append(dataset[num]['lng'])
        naam_kort.append(dataset[num]['namen']['kort'])
        naam_mid.append(dataset[num]['namen']['middel'])
        naam_lang.append(dataset[num]['namen']['lang'])
    num += 1

print(f"{len(naam_kort)} stations, from {naam_mid[0]} to {naam_mid[-1]}, namely:\n{naam_kort}")

397 stations, van Aalten tot Zwijndrecht, namelijk:
['Aalten', 'Abcoude', 'Akkrum', 'Alkmaar', 'Almelo', 'Alkmaar N', 'Alphen', 'Schothorst', 'Amersfrt C', 'RAI', 'Amstel', 'Amsterdm C', 'Sloterdijk', 'Muiderprt', 'Amsterdm Z', 'Anna Paulo', 'Apeldoorn', 'Appingedam', 'Arkel', 'Arnemuiden', 'Arnhem C', 'Velperprt', 'Assen', 'Bijlmer A', 'Presikhaaf', 'Lelylaan', 'Almere C', 'Buiten', 'Muziekwijk', 'Baarn', 'Baflo', 'Barendrcht', 'Barnevld C', 'Barnevld N', 'Bedum', 'Beek-E', 'Beesd', 'Beilen', 'Parkwijk', 'Bergen opZ', 'Best', 'Beverwijk', 'Bilthoven', 'Barnevld Z', 'Blerick', 'Bloemendl', 'Bodegraven', 'Borne', 'Boskoop', 'Snijdelwk', 'Bovenk-Gr', 'Boxmeer', 'Boxtel', 'Breda', 'Prinsenbk', 'Breukelen', 'Lansingerl', 'Brummen', 'Buitenpost', 'Bunde', 'Bunnik', 'Bussum Z', 'Schollevr', 'Castricum', 'Chevremont', 'Coevorden', 'Culemborg', 'Cuijk', 'Dalen', 'Diemen Z', 'Daarlervn', 'Diemen', 'Delft Camp', 'Dalfsen', 'Deinum', 'Delden', 'Delft', 'Delfzijl', 'Deurne', 'Deventer', 'Didam', '

## Step 2: times and destinations

This step was harder, because the data received from the API isn't clean. There were stations without departure times, stations that return an error and stations that were perhaps marked as having departure times, but that had no destination. 
1. I'll start with an empty list and set my number back to 0. 
2. I'll return to my dataset and only start the loop for 'land'='NL'. 
3. For each location I'll check if the 'has departure time' variable is true. If it's not, I'll append my list with 'Geen informatie' (= 'No information'). 
4. If a station should have times, I'll put the stationcode in the URL of the API and request the times and destinations of this station, turning it into JSON.
5. Then I'll see if the object starts with 'payload', and is therefore not an error. If it is an error, again, I will add 'no information'. 
6. If I do receive information, I'll check if there is info in the 'departures' list. If there isn't, again: 'no information'. 
7. If all the above variables are correct, I'll append the next departure time and destination. 

Finally, I'll check if I have the same amount of items in my list as previously collected, and that I have something for each station. 

> It will take approximately 2 minutes to run this code!

In [4]:
eindstation = []
vertrektijd = []
num = 0
for item in dataset:
    if dataset[num]['land'] == 'NL':
        if dataset[num]['heeftVertrektijden'] == True:
            times = requests.get('https://gateway.apiportal.ns.nl/reisinformatie-api/api/v2/departures?station='+dataset[num]['code']+'&maxJourneys=1', headers=my_headers)
            info = times.json()
            if 'payload' in info.keys():
                if len(info['payload']['departures']) == 0:
                    eindstation.append('Geen informatie')
                    vertrektijd.append('Geen informatie')
                elif 'direction' in info['payload']['departures'][0].keys():
                    eindstation.append(info['payload']['departures'][0]['direction'])
                    vertrektijd.append(info['payload']['departures'][0]['plannedDateTime'])
            else: 
                eindstation.append('Geen informatie')
                vertrektijd.append('Geen informatie')
        else: 
            eindstation.append('Geen informatie')
            vertrektijd.append('Geen informatie')
    num +=1
print(f"{len(eindstation)} destinations and {len(vertrektijd)} departure times. For example, at {vertrektijd[0]} to {eindstation[0]}.")


397 bestemmingen en 397 vertrektijden. Bijvoorbeeld om 2022-03-24T13:30:00+0100 naar Arnhem Centraal.


## Step 3: to a dataframe

I'll make a dictionary of all the information I have. Each station will get its corresponding names, code, long/lat, departure time and destination. 

Then I'll turn this dictionary into a dataframe, to display and process it more easily. 

In [5]:
statdict = {'Short': naam_kort, 'Middle': naam_mid, 'Long': naam_lang, 'Code': stationscodes, \
'Longitude': longitudes, 'Latitude': latitudes, 'Next_departure_time': vertrektijd, 'Matching_final_destination': eindstation}

import pandas as pd

df = pd.DataFrame(statdict)

df

,Kort,Middel,Lang,Code,Longitude,Latitude,Volgende vertrektijd,Bijbehorend eindstation
0,Aalten,Aalten,Aalten,ATN,6.578627,51.921327,2022-03-24T13:30:00+0100,Arnhem Centraal
1,Abcoude,Abcoude,Abcoude,AC,4.977000,52.278500,2022-03-24T13:02:00+0100,Driebergen-Zeist
2,Akkrum,Akkrum,Akkrum,AKM,5.843611,53.046391,2022-03-24T13:12:00+0100,Leeuwarden
3,Alkmaar,Alkmaar,Alkmaar,AMR,4.739722,52.637779,2022-03-24T13:04:00+0100,Amsterdam Centraal
4,Almelo,Almelo,Almelo,AML,6.653889,52.358055,2022-03-24T13:01:00+0100,Zwolle
...,...,...,...,...,...,...,...,...
392,Zutphen,Zutphen,Zutphen,ZP,6.194167,52.145279,2022-03-24T13:03:00+0100,Zwolle
393,Westereen,De Westereen,De Westereen,DWE,6.034630,53.248170,2022-03-24T13:08:00+0100,Groningen
394,Stadshagen,Stadshagen,Zwolle Stadshagen,ZLSH,6.051460,52.527640,2022-03-24T13:09:00+0100,Zwolle
395,Zwolle,Zwolle,Zwolle,ZL,6.091944,52.504723,2022-03-24T13:07:00+0100,Amsterdam Centraal


## Step 4: to the database

Now I'll contact my server and prepare the data to be added. 
I'll write code to make a new database and table with the columns as shown above, and to put the data in the right column. 
Ik schrijf de code om een nieuwe database en tabel aan te maken met de bijbehorende kolommen zoals je die hierboven ziet, en de juiste data in de juiste kolom te zetten. 

In [6]:
import mysql.connector

# ADD YOUR LOGIN INFO HERE 
mydb = mysql.connector.connect(
  host="localhost",
  user="username_here",
  password="password_here"
)

cursor = mydb.cursor()

cursor.execute("DROP DATABASE IF EXISTS verzamelen_ns")
cursor.execute("CREATE DATABASE IF NOT EXISTS verzamelen_ns")
cursor.execute("USE verzamelen_ns")
cursor.execute("DROP TABLE IF EXISTS nlstations")
cursor.execute(
    "CREATE TABLE nlstations"
    "(Short VARCHAR(255) NOT NULL, Middle VARCHAR(255) NOT NULL, Long VARCHAR(255) NOT NULL,\
Code VARCHAR(255) NOT NULL, Longitude FLOAT(23) NOT NULL, Latitude FLOAT(23) NOT NULL, Next_departure_time VARCHAR(255) NOT NULL, Matching_final_destination VARCHAR(255) NOT NULL)"
    )

for row in df.itertuples(index=False):
    sql = "INSERT INTO nlstations (Short, Middle, Long, Code, Longitude, Latitude, Next_departure_time, Matching_final_destination)\
values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, row)
    mydb.commit()
print("File loaded succesfully!")


Bestand succesvol geladen!


## Ready for departure!

The information of all stations has been retreived and loaded in the database. It's easy to reach now! 